# Affinity Analysis Sederhana

Common use case dari data mining yaitu untuk meningkatkan penjualan. Ini bisa dilakukan dengan affinity analysis.

## Apa itu Affinity Analysis

Affinity analysis adalah tipe data mining yang memberikan kesamaan antara sampel (objek). Contohnya kesamaan antar objek berikut:

- user pada website, untuk pengiklanan bertarget (targeted ads)
- item/barang yang di berikan/sarankan ke user, untuk memberikan rekomendasi film atau produk
- gen manusia, untuk menemukan orang dengan leluhur yang sama

note: We can measure affinity in a number of ways. For instance, we can record how frequently two products are purchased together. We can also record the accuracy of the statement when a person buys object 1 and also when they buy object 2. Other ways to measure affinity include computing the similarity between samples, which we will cover in later chapters.

## Product Recommendations

Kita akan membuat product recommendation yang berdasar pada ide: ketika dua item dibeli secara bersamaan di masa lalu, kemungkinan besar akan dibeli secara bersamaan lagi di masa depan. 

Untuk menyederhanakan kodingan, kita anggap hanya dua item yang dibeli secara bersamaan, misal: susu dan roti.

> Jika seseorang membeli produk X, maka kemungkinan besar juga akan membeli produk Y

## Load Dataset

Download [data](https://raw.githubusercontent.com/PacktPublishing/Learning-Data-Mining-with-Python/master/Chapter%201/affinity_dataset.txt)

Kemudian import library

In [1]:
import pandas as pd

Load dataset dengan fungsi `read_csv` dari pandas. `header=None` karena datanya tidak memiliki header (nama kolom), `delim_whitespace=True` karena datanya dipisahkan dengan spasi.

In [6]:
# jika dataset sudah didownload gunakan baris kode dibawah
# data = pd.read_csv("lokasi/file/dataset_affinity.txt", header=None, delim_whitespace=True)

# kita juga bisa akses langsung data dari internet
url = "https://raw.githubusercontent.com/PacktPublishing/Learning-Data-Mining-with-Python/master/Chapter%201/affinity_dataset.txt"
data = pd.read_csv(url, header=None, delim_whitespace=True)

Melihat 5 baris pertama pada dataset

In [8]:
data.head(10)

,0,1,2,3,4
0,0,0,1,1,1
1,1,1,0,1,0
2,1,0,1,1,0
3,0,0,1,1,1
4,0,1,0,0,1
5,0,1,0,0,0
6,1,0,0,0,1
7,1,0,0,0,1
8,0,0,0,1,1
9,0,0,1,1,1


Beri nama kolom untuk memudahkan membaca

In [11]:
data.rename(columns={0:"roti", 1:"susu", 2:"keju", 3:"apel", 4:"pisang"},
           inplace=True)

In [12]:
data

,roti,susu,keju,apel,pisang
0,0,0,1,1,1
1,1,1,0,1,0
2,1,0,1,1,0
3,0,0,1,1,1
4,0,1,0,0,1
...,...,...,...,...,...
95,0,1,0,0,0
96,0,0,1,1,0
97,0,1,1,0,0
98,0,0,1,1,0


## Mengimplementasikan Ranking Rules Sederhana

Rules bisa dengan dibuat hanya dengan menemukan sampel yang berisi dua produk dibeli secara bersamaan. Namun kita perlu cara untuk mengukur rules yang bagus dan yang jelek. Dengan ini kita bisa memilih produk spesifik untuk direkomendasikan.

Rules dapat diukur dengan banyak cara, untuk kali ini kita fokus ke **support** dan **confidence**.

- **Support**, adalah jumlah rule muncul dalam dataset, dihitung dari jumlah sampel (baris) dimana rule nya valid.
- **Confidence**, mengukur seberapa akurat rule nya ketika digunakan, dihitung dari persentase rule valid untuk setiap jumlah pembelian premise. Pertama hitung berapa kali rule nya diterapkan pada dataset kemudian dibagi jumlah kejadian premis dibeli

Sebagai contoh, kita akan menghitung support dan confidence untuk rule berikut

>jika orang beli apel, maka juga beli pisang

- premis = apel
- conclusion = pisang

<br></br>
<details>
    <summary>lihat kodingan</summary>
<code>
jml_pembelian_apel = data[data.apel == 1].shape[0]
jml_pembelian_pisang = data[data.apel == 1].shape[0]
</code>
<details

In [38]:
# hitung berapa kali orang membeli Apel <-- premis
# yaitu jumlah baris pada kolom apel yang bernilai 1

# data["apel"] == 1
jml_pembelian_apel = data[data["apel"] == 1].shape[0]

In [36]:
# hitung berapa kali orang membeli Apel <-- premis
# yaitu jumlah baris pada kolom apel yang bernilai 1

jml_pembelian_pisang =  data[data["pisang"] == 1].shape[0]

Hitung kejadian dimana rule valid dan rule tidak valid
<br></br>
<details>
    <summary>lihat kodingan</summary>
<code>
rule_valid = data[(data.apel == 1) & (data.pisang == 1)].shape[0]
rule_invalid = data[(data.apel == 1) & (data.pisang == 0)].shape[0]
</code>
<details>

In [40]:
# hitung kejadian dimana rule valid (beli apel dan pisang) dan rule tidak valid (cuma beli apel)
# orang beli apel dan pisang
rule_valid = data[(data["pisang"] == 1) & (data["apel"] == 1)].shape[0]

# orang beli apel aja (tanpa pisang)
rule_invalid = data[(data["pisang"] == 0) & (data["apel"] == 1)].shape[0]

print(f"Jumlah rule valid {rule_valid}")
print(f"Jumlah rule invalid {rule_invalid}")

Jumlah rule valid 21
Jumlah rule invalid 15


In [42]:
support = rule_valid
confidence = rule_valid / jml_pembelian_apel

print(f"Supportnya adalah {support}")
print("Confidence :", confidence)

Supportnya adalah 21
Confidence : 0.5833333333333334


---

Sekarang kita akan menghitung statistik dari seluruh rules. Pertama kita buat dictionary masing-masing untuk rules valid dan rules tidak valid. `Key` nya adalah `tuple` (premis, conclusion). Jika premis dan conclusion ada pembelian maka rule valid, jika premis saja yang ada pembeliannya maka rule tidak valid.

Untuk menghitung confidence dan support untuk semua kemungkinan rules, kita buat dictionary untuk menampung hasilnya.

`defaultdict` sama kaya dictionary biasa, cuma kalau kita mengakses atau memodifikasi key yang tidak ada, `defaultdict` akan membuatkan key tersebut.

[sumber](https://realpython.com/python-defaultdict/)

In [43]:
from collections import defaultdict

rules_valid = defaultdict(int)
rules_invalid = defaultdict(int)
jml_kemunculan = defaultdict(int)

Selanjutnya kita cek tiap fitur pada tiap sampel dataset

In [49]:
# loop tiap baris (sampel)
for idx_sampel, sampel in data.iterrows():
    # loop tiap item pada baris saat ini
    for premis, pembelian_premis in sampel.items():
        # jika tidak ada pembelian, skip
        if pembelian_premis == 0: 
            continue
        # hitung premis yang dibeli di transaksi lain
        else:
            jml_kemunculan[premis] += 1
        for conclusion, pembelian_conclusion in sampel.items():
            print()
            # jika orang beli apel dan juga beli apel. Kita nda perlu datanya
            if premis == conclusion:
                continue
            
            # jika orang juga beli item conclusion
            if pembelian_conclusion == 1:
                rules_valid[(premis, conclusion)] += 1
            
            # jika orang beli premis tapi nda beli conclusion
            else:
                rules_invalid[(premis, conclusion)] += 1


support = rules_valid
confidence = defaultdict(float)
for premis, conclusion in rules_valid.keys():
    confidence[(premis, conclusion)] = rules_valid[(premis, conclusion)] / jml_kemunculan[premis]

Buat fungsi untuk menampilkan rule

In [52]:
def print_rule(premis, conclusion, support, confidence):
    print(f"Rule: Jika orang beli {premis}, maka juga akan membeli {conclusion}")
    print(f" - Support : {support[(premis, conclusion)]}")
    print(f" - Confidence : {confidence[(premis, conclusion)]}\n")

In [54]:
premis = "pisang"
conclusion = "apel"
print_rule(premis, conclusion, support, confidence)

Rule: Jika orang beli pisang, maka juga akan membeli apel
 - Support : 21
 - Confidence : 0.3559322033898305



## Ranking Rules Terbaik

Pertama urutkan dictionary `support`. Fungsi `items()` mengembalikan list isi dictionary dalam bentuk tuple _(key, value)_. Kemudian kita urutkan list ini dengan class `itemgetter` sebagai key-nya. `itemgetter(1)` berarti urutkan berdasarkan _value_-nya. `reverse=True` berarti nilai tertinggi dulu.

In [56]:
from operator import itemgetter
support_terurut = sorted(support.items(), key=itemgetter(1), reverse=True)

support_terurut

[(('keju', 'pisang'), 27),
 (('pisang', 'keju'), 27),
 (('keju', 'apel'), 25),
 (('apel', 'keju'), 25),
 (('apel', 'pisang'), 21),
 (('pisang', 'apel'), 21),
 (('susu', 'pisang'), 19),
 (('pisang', 'susu'), 19),
 (('roti', 'pisang'), 17),
 (('pisang', 'roti'), 17),
 (('roti', 'susu'), 14),
 (('susu', 'roti'), 14),
 (('susu', 'apel'), 9),
 (('apel', 'susu'), 9),
 (('susu', 'keju'), 7),
 (('keju', 'susu'), 7),
 (('roti', 'apel'), 5),
 (('apel', 'roti'), 5),
 (('roti', 'keju'), 4),
 (('keju', 'roti'), 4)]

In [58]:
for index in range(5):
    print(f"Rule #{index + 1}")
    premis, conclusion = support_terurut[index][0]
    print_rule(premis, conclusion, support, confidence)

Rule #1
Rule: Jika orang beli keju, maka juga akan membeli pisang
 - Support : 27
 - Confidence : 0.6585365853658537

Rule #2
Rule: Jika orang beli pisang, maka juga akan membeli keju
 - Support : 27
 - Confidence : 0.4576271186440678

Rule #3
Rule: Jika orang beli keju, maka juga akan membeli apel
 - Support : 25
 - Confidence : 0.6097560975609756

Rule #4
Rule: Jika orang beli apel, maka juga akan membeli keju
 - Support : 25
 - Confidence : 0.6944444444444444

Rule #5
Rule: Jika orang beli apel, maka juga akan membeli pisang
 - Support : 21
 - Confidence : 0.5833333333333334



Selanjutnya buat print_rule untuk confidence
<br></br>

<details>
    <summary>lihat kodingan</summary>
<code>
confidence_terurut = sorted(confidence.items(), key=itemgetter(1), reverse=True)

for index in range(5):
    print(f"Rule #{index + 1}")
    premis, conclusion = confidence_terurut[index][0]
    print_rule(premis, conclusion, support, confidence)
</code>
</details>

In [59]:
confidence_terurut = sorted(confidence.items(), key=itemgetter(1), reverse=True)

for index in range(5):
    print(f"Rule #{index + 1}")
    premis, conclusion = confidence_terurut[index][0]
    print_rule(premis, conclusion, support, confidence)
    

Rule #1
Rule: Jika orang beli apel, maka juga akan membeli keju
 - Support : 25
 - Confidence : 0.6944444444444444

Rule #2
Rule: Jika orang beli keju, maka juga akan membeli pisang
 - Support : 27
 - Confidence : 0.6585365853658537

Rule #3
Rule: Jika orang beli roti, maka juga akan membeli pisang
 - Support : 17
 - Confidence : 0.6296296296296297

Rule #4
Rule: Jika orang beli keju, maka juga akan membeli apel
 - Support : 25
 - Confidence : 0.6097560975609756

Rule #5
Rule: Jika orang beli apel, maka juga akan membeli pisang
 - Support : 21
 - Confidence : 0.5833333333333334

